In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [10]:
!pip install transformers peft accelerate bitsandbytes datasets trl -q --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 10.7 MB/s eta 0:00:00


In [6]:
from datasets import load_dataset

# Load the CSV file (adjust the file path as needed)
dataset = load_dataset("csv", data_files={"train": "/kaggle/input/synthetic-qa-dataset-based-on-ai-research-papers/combined_alpaca_dataset.csv"})["train"]

# Check existing columns (expecting at least 'instruction' and 'output')
print(dataset.column_names)

# If the dataset lacks an "input" column, add one with empty strings:
def add_empty_input(example):
    if "input" not in example:
        example["input"] = ""
    return example

dataset = dataset.map(add_empty_input)
print("Columns after update:", dataset.column_names)


Generating train split: 0 examples [00:00, ? examples/s]

['instruction', 'output', 'source', 'metadata']


Map:   0%|          | 0/264 [00:00<?, ? examples/s]

Columns after update: ['instruction', 'output', 'source', 'metadata', 'input']


In [13]:
split_dataset = dataset.train_test_split(test_size=0.1, seed=49)
train_dataset = split_dataset["train"]
val_dataset = split_dataset["test"]

print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")


Training set size: 237
Validation set size: 27


In [14]:
train_dataset[7]

{'instruction': 'Explain how DeepSeek\'s DualPipe addresses the "bubble" problem in distributed GPU training, and analyze the trade-offs between communication reduction and potential synchronization overhead introduced by processing batches in opposite directions.',
 'output': 'DeepSeek\'s DualPipe addresses the "bubble" problem, a training inefficiency where GPUs remain idle waiting for data, by overlapping forward and backward passes. In conventional distributed training, GPUs process data sequentially, leading to idle time (the "bubble") while waiting for data from preceding GPUs.\n\nDualPipe innovatively initiates training data from two devices in opposite directions. While device 0 processes batch 0 in the forward pass, device 7 (in a system with 8 GPUs) starts processing a different batch in the reverse direction. This concurrent processing reduces idle time. Instead of device 7 waiting for batch 0, it immediately begins computation. The forward and backward passes are combined a

In [15]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Replace with your chosen instruct model id; for example, using Qwen2.5-3B-Instruct
model_id = "Qwen/Qwen2.5-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
# Ensure the pad token is set (often to the EOS token)
tokenizer.pad_token = tokenizer.eos_token

# Load the model (without quantization in this case; plain LoRA is sufficient)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16)


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.6k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [16]:
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

# Enable gradient checkpointing (optional but useful for memory saving)
model.gradient_checkpointing_enable()

# For plain LoRA, we don’t need to quantize the base model further
# Prepare the model for adapter training (freezes most base parameters)
# (prepare_model_for_kbit_training is typically used for QLoRA; for plain LoRA, you may skip it,
# but using it doesn't hurt if your base model is large)
model = prepare_model_for_kbit_training(model)

# Set up LoRA configuration:
peft_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=16,                  # Low-rank dimension (try 16 or 32; lower values are safer for small datasets)
    lora_alpha=32,         # Scaling factor for adapter updates
    lora_dropout=0.1,      # Dropout for adapter layers to reduce overfitting
    target_modules=["q_proj", "v_proj"],  # Typical target modules in the Transformer
    bias="none"
)

# Apply LoRA to the model:
lora_model = get_peft_model(model, peft_config)
lora_model.print_trainable_parameters()


trainable params: 3,686,400 || all params: 3,089,625,088 || trainable%: 0.1193


In [19]:
def tokenize_function(batch):
    prompts = []
    for instruction, inp, output in zip(batch["instruction"], batch["input"], batch["output"]):
        # Check if the input is empty after stripping whitespace.
        if inp.strip() == "":
            prompt = (
                "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n"
                f"### Instruction:\n{instruction}\n\n"
                f"### Response:\n{output}"
            )
        else:
            prompt = (
                "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n"
                f"### Instruction:\n{instruction}\n\n"
                f"### Input:\n{inp}\n\n"
                f"### Response:\n{output}"
            )
        prompts.append(prompt)
    # Tokenize all prompts together
    return tokenizer(prompts, truncation=True, max_length=512, padding="max_length")
    
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)

# Optionally inspect a sample tokenized output:
print(tokenized_train[7])


Map:   0%|          | 0/237 [00:00<?, ? examples/s]

{'instruction': 'Explain how DeepSeek\'s DualPipe addresses the "bubble" problem in distributed GPU training, and analyze the trade-offs between communication reduction and potential synchronization overhead introduced by processing batches in opposite directions.', 'output': 'DeepSeek\'s DualPipe addresses the "bubble" problem, a training inefficiency where GPUs remain idle waiting for data, by overlapping forward and backward passes. In conventional distributed training, GPUs process data sequentially, leading to idle time (the "bubble") while waiting for data from preceding GPUs.\n\nDualPipe innovatively initiates training data from two devices in opposite directions. While device 0 processes batch 0 in the forward pass, device 7 (in a system with 8 GPUs) starts processing a different batch in the reverse direction. This concurrent processing reduces idle time. Instead of device 7 waiting for batch 0, it immediately begins computation. The forward and backward passes are combined as

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq

# Updated training arguments (note: you can switch to eval_strategy in newer versions)
training_args = TrainingArguments(
    output_dir="./output",                   # Directory for saving model checkpoints.
    per_device_train_batch_size=2,           # Small batch size to manage memory.
    per_device_eval_batch_size=2,            # Small eval batch size.
    gradient_accumulation_steps=4,           # Simulate a larger effective batch size.
    num_train_epochs=5,                      # More epochs might be needed for a small dataset.
    learning_rate=2e-4,                      # Starting learning rate.
    weight_decay=0.0,                        # No weight decay; adjust if overfitting occurs.
    warmup_steps=20,                         # Short warmup period.
    logging_steps=10,                        # Log training metrics every 10 steps.
    evaluation_strategy="steps",             # Evaluate every eval_steps.
    eval_steps=20,                           # Evaluation frequency.
    save_strategy="steps",                   # Save checkpoint every few steps.
    save_steps=20,                           # Checkpoint frequency.
    save_total_limit=2,                      # Keep only a couple of checkpoints.
    fp16=True,                               # Enable mixed precision for faster training.
    load_best_model_at_end=True,             # Load best model based on eval loss.
    run_name="my_unique_run_name",           # Distinct run name to avoid wandb warnings.
)

# Data collator for dynamic padding:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=lora_model, padding=True)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
)

# Disable cache to save memory during training.
lora_model.config.use_cache = False

# Start training:
trainer.train()

# Save the fine-tuned model and tokenizer:
lora_model.save_pretrained("output/finetuned_model")
tokenizer.save_pretrained("output/finetuned_model")


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


<IPython.core.display.Javascript object>

In [ ]:
# Example prompt for inference:
test_prompt = (
    "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n"
    "### Instruction:\nWhat is the meaning of life?\n\n"
    "### Response:\n"
)

# Tokenize the test prompt:
inputs = tokenizer(test_prompt, return_tensors="pt", padding=True, truncation=True).to("cuda")

# Generate response (adjust max_new_tokens as needed):
with torch.no_grad():
    outputs = lora_model.generate(inputs["input_ids"], max_new_tokens=50)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


In [ ]:
# Merge the LoRA weights into the base model.
# (This function is provided by the PEFT library.)
merged_model = lora_model.merge_and_unload()

# Save the merged model for later use.
merged_model.save_pretrained("output/qwen_ai_research_qa_final")


In [ ]:
from transformers import BitsAndBytesConfig

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,                # Enable 4-bit quantization.
    bnb_4bit_quant_type="nf4",        # Use the NormalFloat 4 (NF4) quantization type.
    bnb_4bit_use_double_quant=True,   # Activate double quantization for additional precision.
    bnb_4bit_compute_dtype=torch.bfloat16  # Use bfloat16 for computations during forward/backward passes.
)


In [ ]:
from transformers import AutoModelForCausalLM

quantized_model = AutoModelForCausalLM.from_pretrained(
    "output/merged_model", 
    quantization_config=quant_config, 
    torch_dtype=torch.bfloat16  # Ensure compatibility with your compute dtype.
)

# Save the quantized model for future inference.
quantized_model.save_pretrained("output/qwen_ai_research_qa_final_Q4")


In [ ]:
# Prepare an inference prompt.
test_prompt = (
    "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n"
    "### Instruction:\nWhat is the meaning of life?\n\n"
    "### Response:\n"
)

inputs = tokenizer(test_prompt, return_tensors="pt", padding=True, truncation=True).to("cuda")

with torch.no_grad():
    outputs = quantized_model.generate(inputs["input_ids"], max_new_tokens=50)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))
